In [1]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import traceback

In [2]:
emotion_model_name = "SamLowe/roberta-base-go_emotions"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_name)
emotion_labels = emotion_model.config.id2label

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [3]:
mh_model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
mh_tokenizer = AutoTokenizer.from_pretrained(mh_model_name)
mh_model = AutoModelForSequenceClassification.from_pretrained(mh_model_name)
mh_labels = mh_model.config.id2label

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
def detect_emotion(text):
    try:
        inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = emotion_model(**inputs)

        probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
        top_idx = int(np.argmax(probs))

        top_emotion = emotion_labels[top_idx]
        scores = {emotion_labels[i]: float(probs[i]) for i in range(len(probs))}

        return top_emotion, scores

    except Exception:
        return f"Emotion Model Error:\n{traceback.format_exc()}", {}


In [5]:
def detect_mental_health(text):
    try:
        inputs = mh_tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = mh_model(**inputs)

        probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
        top_idx = int(np.argmax(probs))

        top_issue = mh_labels[top_idx]
        scores = {mh_labels[i]: float(probs[i]) for i in range(len(probs))}

        return top_issue, scores

    except Exception:
        return f"Mental Health Model Error:\n{traceback.format_exc()}", {}



In [6]:
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Emotion + Mental-Health Detector")

    text_in = gr.Textbox(label="Enter text")

    gr.Markdown("### Emotion Model Output")
    emo_label = gr.Textbox(label="Predicted Emotion")
    emo_scores = gr.JSON(label="Emotion Probabilities")

    gr.Markdown("### Mental-Health Classification Output")
    mh_label = gr.Textbox(label="Mental Health Category")
    mh_scores = gr.JSON(label="Mental-Health Probabilities")

    btn = gr.Button("Analyze")

    btn.click(
        fn=lambda t: (
            *detect_emotion(t),
            *detect_mental_health(t)
        ),
        inputs=text_in,
        outputs=[emo_label, emo_scores, mh_label, mh_scores]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33456c3d9b108a52d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
